In [1]:
import os
import numpy as np
import pandas as pd
import swifter
from sklearn.model_selection import train_test_split
import utils
from hep_ml.nnet import MLPClassifier

Using cuDNN version 6021 on context None
Mapped name None to device cuda: GeForce GTX TITAN X (0000:02:00.0)


In [2]:
DATA_PATH = "./data"

In [3]:
train, test = utils.load_data_csv(DATA_PATH, utils.SIMPLE_FEATURE_COLUMNS)

/root/anaconda3/envs/idao_cb/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
func = utils.find_closest_hit_per_station

In [5]:
# dt_train = pd.read_hdf('closest_hits_features.train.filled.p1000.hdf', 'key')
# dt_test  = pd.read_hdf('closest_hits_features.test.filled.p1000.hdf', 'key')

dt_train = pd.read_hdf('data/closest_hits_features.train.filled.m9999.v1.hdf', 'key')
dt_test  = pd.read_hdf('data/closest_hits_features.test.filled.m9999.v1.hdf', 'key')

In [6]:
dt_train.columns = [func.__name__ + '_' + str(c) if str(c).isnumeric() else c for c in dt_train.columns]
dt_test.columns  = [func.__name__ + '_' + str(c) if str(c).isnumeric() else c for c in dt_test.columns]

In [7]:
train = pd.concat([train, dt_train], axis=1, copy=False)
test = pd.concat([test, dt_test], axis=1, copy=False)

In [8]:
train.shape, test.shape

((5445705, 115), (726095, 113))

In [9]:
set(train.columns.tolist()).symmetric_difference(test.columns.tolist())

{'label', 'weight'}

In [10]:
del train

In [11]:
train = test

## Новые признаки

In [12]:
NA_VALUE = -9999.0

In [13]:
def fillna(df):
    """
    Пропуски только в 2 и 3 колонках
    """
    
    for axis in 'XYZ':
        cols = ['MatchedHit_{}[{}]'.format(axis, i) for i in range(4)]
        mask = np.isclose(df.loc[:, cols[2]], NA_VALUE)
        mask = mask & ~np.isclose(df.loc[:, cols[3]], NA_VALUE)
        
        df.loc[mask, cols[2]] = df.loc[mask, [cols[1], cols[3]]].mean(axis=1)
    return df

In [14]:
# train, test = map(fillna, [train, test])

In [15]:
train = fillna(train)

### Модифицируем matched_hit_type

In [16]:
def modify_matched_hit_type(df):
    map_type = {2: 1, 0: 0, 1: -1}
    cols = df.columns[df.columns.str.startswith('MatchedHit_TYPE')]
    for col in cols:
        df[col] = df[col].map(map_type)
    return df

In [17]:
# train, test = map(modify_matched_hit_type, [train, test])

In [18]:
train = modify_matched_hit_type(train)

### Вычесть среднее из MatchedHit_Z

In [19]:
matched_hit_z_cols = train.columns[train.columns.str.startswith("MatchedHit_Z")]
matched_hit_z_mean = pd.Series([15270, 16470, 17670, 18870], index=matched_hit_z_cols)
matched_hit_z_mean

MatchedHit_Z[0]    15270
MatchedHit_Z[1]    16470
MatchedHit_Z[2]    17670
MatchedHit_Z[3]    18870
dtype: int64

In [20]:
def make_unbiased_z(df):
    for i in range(4):
        distance = np.zeros(df.shape[0])
    
        col = f"MatchedHit_Z[{i}]"
        mask = np.isclose(df.loc[:, col], NA_VALUE)
        distance[~mask] = df.loc[~mask, col] - matched_hit_z_mean[col]
        distance[mask] = 0
    
        df[col] = distance
        
        df[f"MatchedHit_Z[{i}]_sign"] = np.sign(df[col]).astype(np.int8)
    return df

In [21]:
# train, test = map(make_unbiased_z, [train, test])

In [22]:
train = make_unbiased_z(train)

In [23]:
train.loc[:, train.columns.str.startswith("MatchedHit_Z")].head()

,MatchedHit_Z[0],MatchedHit_Z[1],MatchedHit_Z[2],MatchedHit_Z[3],MatchedHit_Z[0]_sign,MatchedHit_Z[1]_sign,MatchedHit_Z[2]_sign,MatchedHit_Z[3]_sign
0,126.441,-65.305,-72.188,-63.771,1,-1,-1,-1
1,-152.701,-148.568,-155.830,-147.787,-1,-1,-1,-1
2,-157.764,-154.032,-161.697,-154.055,-1,-1,-1,-1
3,-150.160,-145.826,-152.887,56.592,-1,-1,-1,1
4,-66.947,-62.676,-69.800,-61.620,-1,-1,-1,-1


### Расстояние до центра (+)

In [24]:
def compute_polar_distances_matched_hit(df):
    for i in range(4):
        cols = [f"MatchedHit_X[{i}]", f"MatchedHit_Y[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)
    
        name = f"MatchedHit_R[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = np.log(1.0 + np.power(df.loc[~mask, cols], 2).sum(axis=1))
        
        name = f"MatchedHit_A[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = np.arctan2(df.loc[~mask, cols[1]], df.loc[~mask, cols[0]])

    # намеренное переупорядочивание признаков
    cols  = df.columns[df.columns.str.startswith("MatchedHit_R")].tolist()
    cols += df.columns[df.columns.str.startswith("MatchedHit_A")].tolist()
    cols  = df.columns[:-len(cols)].tolist() + cols
    df = df[cols]

    return df

In [25]:
# train, test = map(compute_polar_distances_matched_hit, [train, test])

In [26]:
train = compute_polar_distances_matched_hit(train)

In [27]:
train.loc[:, train.columns.str.startswith("MatchedHit_R")].head()

,MatchedHit_R[0],MatchedHit_R[1],MatchedHit_R[2],MatchedHit_R[3]
0,15.045311,15.028344,15.170549,15.302802
1,13.993879,14.148052,14.229975,14.363091
2,16.444550,16.597730,16.726557,16.858928
3,12.540267,12.700585,12.846768,13.174559
4,12.503433,12.698478,12.888758,13.025454


In [28]:
train.loc[:, train.columns.str.startswith("MatchedHit_A")].head()

,MatchedHit_A[0],MatchedHit_A[1],MatchedHit_A[2],MatchedHit_A[3]
0,-1.582870,-1.563293,-1.540319,-1.539549
1,-2.936887,-2.936216,-2.930155,-2.931664
2,-2.688974,-2.688812,-2.685729,-2.686143
3,1.986865,1.999996,2.003493,1.986901
4,2.606379,2.621242,2.634569,2.633024


In [29]:
def replace_na(df):
    df[df == NA_VALUE] = None
    df.fillna(df.mean(), inplace=True)
    return df

In [30]:
# train, test = map(replace_na, [train, test])

In [31]:
train = replace_na(train)

### Дельты и отношения для радиусов (+)

In [32]:
def ratio_radius_matched_hit(df):
    for i, j in zip([0, 1, 2, 3], [1, 2, 3, 0]):
        col_i = f"MatchedHit_R[{i}]"
        col_j = f"MatchedHit_R[{j}]"
        mask = np.isclose(df.loc[:, [col_i, col_j]], NA_VALUE).any(axis=1)

        name = f"MatchedHit_R[{j}/{i}]_delta"
        name_abs = name + '_abs'
        df[name] = df[name_abs] = NA_VALUE
        
        df.loc[~mask, name] = df.loc[~mask, col_j] - df.loc[~mask, col_i]
        df.loc[~mask, name_abs] = np.abs(df.loc[~mask, name])
        
        name = f"MatchedHit_R[{j}/{i}]_frac"
        df[name] = NA_VALUE
        df.loc[~mask, name] = df.loc[~mask, col_j] / df.loc[~mask, col_i]
    return df

In [33]:
# train, test = map(ratio_radius_matched_hit, [train, test])

In [34]:
train = ratio_radius_matched_hit(train)

In [35]:
cols = filter(lambda s: s.startswith("MatchedHit_R") and ("_delta" in s or "_frac" in s), train.columns)
train.loc[:, cols].head()

,MatchedHit_R[1/0]_delta,MatchedHit_R[1/0]_delta_abs,MatchedHit_R[1/0]_frac,MatchedHit_R[2/1]_delta,MatchedHit_R[2/1]_delta_abs,MatchedHit_R[2/1]_frac,MatchedHit_R[3/2]_delta,MatchedHit_R[3/2]_delta_abs,MatchedHit_R[3/2]_frac,MatchedHit_R[0/3]_delta,MatchedHit_R[0/3]_delta_abs,MatchedHit_R[0/3]_frac
0,-0.016967,0.016967,0.998872,0.142205,0.142205,1.009462,0.132253,0.132253,1.008718,-0.257491,0.257491,0.983174
1,0.154173,0.154173,1.011017,0.081923,0.081923,1.005790,0.133116,0.133116,1.009355,-0.369211,0.369211,0.974294
2,0.153180,0.153180,1.009315,0.128827,0.128827,1.007762,0.132371,0.132371,1.007914,-0.414378,0.414378,0.975421
3,0.160318,0.160318,1.012784,0.146183,0.146183,1.011510,0.327791,0.327791,1.025515,-0.634292,0.634292,0.951855
4,0.195045,0.195045,1.015599,0.190280,0.190280,1.014984,0.136696,0.136696,1.010606,-0.522021,0.522021,0.959923


### Расстояние между Lextra и MatchedHit (+)

In [36]:
def distance_dist_lextra_matched(df):
    distance_counts = np.zeros(df.shape[0], dtype=int)
    
    distance_se = np.zeros(df.shape[0])
    distance_ae = np.zeros(df.shape[0])
    
    for i in range(4):
        distance = np.zeros(df.shape[0])

        cols = [f"MatchedHit_X[{i}]", f"MatchedHit_Y[{i}]", f"Lextra_X[{i}]", f"Lextra_Y[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)

        mse = df.loc[~mask, cols[:2]].values - df.loc[~mask, cols[2:]].values
        mse = np.power(mse, 2.0).sum(axis=1)
        
        mae = df.loc[~mask, cols[:2]].values - df.loc[~mask, cols[2:]].values
        mae = np.abs(mae).max(axis=1)

        distance[~mask] = np.log(1.0 + mse)
        distance[mask] = NA_VALUE
        
        distance_se[~mask] += mse
        distance_ae[~mask] = np.maximum(distance_ae[~mask], mae)
        distance_counts[~mask] += 1

        df[f"distance_LextraMatchedHit[{i}]"] = distance

#     distance_se /= distance_counts
#     df[f"distance_LextraMatchedHit_SE"] = distance_se
#     df[f"distance_LextraMatchedHit_AE"] = distance_ae

    return df

In [37]:
# train, test = map(distance_dist_lextra_matched, [train, test])

In [38]:
train = distance_dist_lextra_matched(train)

In [39]:
train.loc[:, train.columns.str.startswith("distance_LextraMatchedHit")].head()

,distance_LextraMatchedHit[0],distance_LextraMatchedHit[1],distance_LextraMatchedHit[2],distance_LextraMatchedHit[3]
0,7.905678,9.410019,9.709775,10.382541
1,7.149215,7.209501,7.683212,7.697716
2,7.281535,8.339558,9.620339,10.037943
3,5.786357,6.766075,7.648479,8.442012
4,2.865971,4.313771,5.827522,5.261681


### Отношение дисперсий

In [40]:
def ratio_mextra_matched(df):    
    for i in range(4):
        cols = [f"MatchedHit_DX[{i}]", f"MatchedHit_DY[{i}]", f"Mextra_DX2[{i}]", f"Mextra_DY2[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)

        tmpX = df.loc[~mask, cols[::2]]
        tmpY = df.loc[~mask, cols[1::2]]
        
        ratio = tmpX.iloc[:, 0] / np.sqrt(tmpX.iloc[:, 1]) + \
                tmpY.iloc[:, 0] / np.sqrt(tmpY.iloc[:, 1])

        name = f"ratio_MextraMatchedHit[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = ratio

    return df

In [41]:
# train, test = map(ratio_mextra_matched, [train, test])

In [42]:
train = ratio_mextra_matched(train)

In [43]:
train.loc[:, train.columns.str.startswith("ratio_MextraMatchedHit")].head()

,ratio_MextraMatchedHit[0],ratio_MextraMatchedHit[1],ratio_MextraMatchedHit[2],ratio_MextraMatchedHit[3]
0,2.180889,1.391761,1.516274,1.159187
1,8.071184,5.123200,5.566834,4.322200
2,13.092113,8.331748,8.906781,6.837303
3,2.210887,1.401526,1.520892,2.348935
4,5.097199,3.263563,3.561245,2.762397


### Еще один момент

In [44]:
def momentum(df):
    mask = np.isclose(df.loc[:, ['P', 'PT']], NA_VALUE).any(axis=1)
    df['PO'] = NA_VALUE
    df.loc[~mask, 'PO'] = np.power(df.loc[~mask, 'P'], 2.0) - np.power(df.loc[~mask, 'PT'], 2.0)
    return df

In [45]:
# train, test = map(momentum, [train, test])

In [46]:
train = momentum(train)

In [47]:
train.loc[:, ['P', 'PT', 'PO']].head()

,P,PT,PO
0,6884.502272,1174.484530,4.601696e+07
1,48769.526891,4344.639775,2.359591e+09
2,20296.048412,4109.805750,3.950391e+08
3,26062.046122,940.242439,6.783462e+08
4,59872.343332,1386.377851,3.582775e+09


### Усредняем дисперсии

In [48]:
def mean_equals(df):
    for n in [0, 1, 2, 3]:
        df[f'Mextra_DXY2[{n}]_mean'] = sum([df[f'Mextra_D{k}2[{n}]'] for k in ('X', 'Y')])/2
        df[f'MatchedHit_DXYZ[{n}]_mean'] = sum([df[f'MatchedHit_D{k}[{n}]'] for k in ('X', 'Y', 'Z')])/3
        df.drop(columns=[f'Mextra_D{k}2[{n}]' for k in ('X', 'Y')], inplace=True)
        df.drop(columns=[f'MatchedHit_D{k}[{n}]' for k in ('X', 'Y', 'Z')], inplace=True)
    return df

In [49]:
# train, test = map(mean_equals, [train, test])

In [50]:
train = mean_equals(train)

### Удаляем лишние колонки

In [51]:
import operator
from functools import reduce

def drop_columns(df):
    mask = [df.columns.str.startswith('MatchedHit_X'),
            df.columns.str.startswith('MatchedHit_Y'),
            df.columns.str.startswith('Lextra_X'),
            df.columns.str.startswith('Lextra_Y'),
            df.columns.str.startswith('FOI_hits_N['),
            df.columns.str.startswith('ndof'),
            df.columns.str.startswith('MatchedHit_DT'),
            df.columns.str.startswith('MatchedHit_T'),
            df.columns.str.endswith('sign'),
            df.columns.str.endswith('abs'),
            df.columns.str.endswith('delta')]
    mask = reduce(operator.or_, mask)
    cols = df.columns[mask].tolist()
    cols += df.columns[df.columns.str.startswith('find_closest_hit_per_station')].tolist()
    
    df.drop(columns=cols, inplace=True)
    return df

In [52]:
# train, test = map(drop_columns, [train, test])

In [53]:
train = drop_columns(train)

## Предсказание модели

I know this is incorrect. See it as a low-hanging fruit to beat the baseline. Catboost affirmatively [refused](https://github.com/catboost/catboost/pull/399) to support negative weights. At the same time, its evaluation is [extremly fast](https://catboost.ai/news/best-in-class-inference-and-a-ton-of-speedups), so we are using it as the timing benchmark for Track 2. Feel free to use a patched version with disabled negative weights check.

In [54]:
features = list(filter(lambda s: s not in utils.TRAIN_COLUMNS, train.columns))
features

['ncl[0]',
 'ncl[1]',
 'ncl[2]',
 'ncl[3]',
 'avg_cs[0]',
 'avg_cs[1]',
 'avg_cs[2]',
 'avg_cs[3]',
 'MatchedHit_Z[0]',
 'MatchedHit_Z[1]',
 'MatchedHit_Z[2]',
 'MatchedHit_Z[3]',
 'NShared',
 'FOI_hits_N',
 'PT',
 'P',
 'MatchedHit_R[0]',
 'MatchedHit_R[1]',
 'MatchedHit_R[2]',
 'MatchedHit_R[3]',
 'MatchedHit_A[0]',
 'MatchedHit_A[1]',
 'MatchedHit_A[2]',
 'MatchedHit_A[3]',
 'MatchedHit_R[1/0]_frac',
 'MatchedHit_R[2/1]_frac',
 'MatchedHit_R[3/2]_frac',
 'MatchedHit_R[0/3]_frac',
 'distance_LextraMatchedHit[0]',
 'distance_LextraMatchedHit[1]',
 'distance_LextraMatchedHit[2]',
 'distance_LextraMatchedHit[3]',
 'ratio_MextraMatchedHit[0]',
 'ratio_MextraMatchedHit[1]',
 'ratio_MextraMatchedHit[2]',
 'ratio_MextraMatchedHit[3]',
 'PO',
 'Mextra_DXY2[0]_mean',
 'MatchedHit_DXYZ[0]_mean',
 'Mextra_DXY2[1]_mean',
 'MatchedHit_DXYZ[1]_mean',
 'Mextra_DXY2[2]_mean',
 'MatchedHit_DXYZ[2]_mean',
 'Mextra_DXY2[3]_mean',
 'MatchedHit_DXYZ[3]_mean']

In [55]:
test = train

In [56]:
# Заглушка для scaler

import copy

from sklearn.base import TransformerMixin

class IdentityScaler(TransformerMixin):
    def __init__(self):        
        pass
    
    def fit(self, X, y=None, **fit_params):        
        return self
        
    def transform(self, X, y=None):
        return X
    
    def fit_transform(self, X, y=None, **fit_params):
        return self.transform(X, y)
    
    def get_params(self, deep=True):
        return copy.deepcopy(self.__dict__) if deep else self.__dict__
    
    def set_params(self, **params):
        self.__dict__.update(copy.deepcopy(params))

In [57]:
import pickle

params = pickle.load(open('models/scaler_params.pkl', 'rb'))

In [58]:
for col in params.keys():
    mean, std = params[col]
    
    scale_mean_std = lambda x: (x - mean) / std
#     train[col], test[col] = map(scale_mean_std, [train[col], test[col]])
    test[col] = scale_mean_std(test[col])

In [59]:
models_g = [pickle.load(open('models/hepmlp_model_{}.pkl'.format(i), 'rb'))
            for i in range(5)]

In [60]:
predictions = [model.predict_proba(test.loc[:, features].values)[:, 1] for model in models_g]
predictions = np.asarray(predictions).T

In [61]:
df_pred = pd.DataFrame(predictions,
                       columns=["hepmlp_model_{}".format(i) for i in range(predictions.shape[1])],
                       index=test.index)
df_pred.to_csv("preds/hepmlp_test_preds.csv", sep=',', index=False)
df_pred.head()

,hepmlp_model_0,hepmlp_model_1,hepmlp_model_2,hepmlp_model_3,hepmlp_model_4
0,0.945492,0.962090,0.976482,0.930198,0.968682
1,0.631330,0.743633,0.823962,0.647144,0.789108
2,0.910721,0.864242,0.738441,0.820551,0.782217
3,0.806772,0.823634,0.919691,0.914611,0.818540
4,0.960117,0.906450,0.950518,0.948546,0.946081


In [62]:
df_pred = pd.DataFrame(df_pred.mean(axis=1))
df_pred.columns = ["prediction"]
df_pred.to_csv("hepmlp_submission_vbugaevskii_mean.csv", index_label=utils.ID_COLUMN, sep=',')
df_pred.head()

,prediction
0,0.956589
1,0.727035
2,0.823234
3,0.856649
4,0.942342
